In [8]:
import pandas as pd
import pandas_datareader.data as web
import io
import requests
import time
import datetime

In [9]:
#function to get stock data
def google_stocks(symbol, start, end):
    return web.DataReader(symbol, 'yahoo', start, end)

In [61]:
#get 7 year stock data for Apple
startDate = datetime.datetime(2010, 1, 4)
endDate = datetime.date.today()
stockData = google_stocks('AAPL', startDate, endDate)

In [62]:
def count_missing(dataframe):
    """
    count number of NaN in dataframe
    """
    return (dataframe.shape[0] * dataframe.shape[1]) - dataframe.count().sum()

In [63]:
idx = pd.date_range(startDate, endDate)

stockData.index = pd.DatetimeIndex(stockData.index)
stockData = stockData.reindex(idx, fill_value='np.nan')

In [64]:
stockData.head(10)

,Open,High,Low,Close,Adj Close,Volume
2010-01-04,30.49,30.6429,30.34,30.5729,27.5051,123432400
2010-01-05,30.6571,30.7986,30.4643,30.6257,27.5526,150476200
2010-01-06,30.6257,30.7471,30.1071,30.1386,27.1143,138040000
2010-01-07,30.25,30.2857,29.8643,30.0829,27.0642,119282800
2010-01-08,30.0429,30.2857,29.8657,30.2829,27.2442,111902700
2010-01-09,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
2010-01-10,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
2010-01-11,30.4,30.4286,29.7786,30.0157,27.0038,115557400
2010-01-12,29.8843,29.9671,29.4886,29.6743,26.6967,148614900
2010-01-13,29.6957,30.1329,29.1571,30.0929,27.0732,151473000


In [65]:
#convert the columns to numeric
def convert_to_numeric(dataframe):
    for col in dataframe:
        dataframe[col] = pd.to_numeric(dataframe[col], errors='coerce')
    return dataframe

In [66]:
stockDataNumeric = convert_to_numeric(stockData)

In [75]:
def interpolate(dataframe):
    features = list(dataframe)
#     print (features)
    for feature in features:
#         dataframe['feature'] = dataframe['feature'].interpolate()
        print (feature)
#     return dataframe

In [76]:
# stockDataInterpolated = interpolate(stockDataNumeric)
interpolate(stockDataNumeric)

Open
High
Low
Close
Adj Close
Volume
